In [1]:
import pandas as pd
import subprocess
import json
import requests
import os
import git
import shutil
from tqdm.auto import tqdm
from subprocess import Popen, PIPE, STDOUT
import xml.etree.ElementTree as ET
from guesslang import Guess
import pickle

C:\Users\Grazia\anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (5.1.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
repositories_final = pd.read_csv("../Repositories/repositories_final.csv")

In [5]:
posts = pd.read_csv("../1-Vulnerability_Detection/Results/results_static_analysis_v2.csv")
posts = posts.fillna('')

In [6]:
posts_vulnerable = posts.loc[posts['severity']!='']

In [7]:
files = pd.unique(posts_vulnerable['file'])

In [8]:
len(files)

128227

In [13]:
guess = Guess()

for file in tqdm(files):
    name_to_save = file.split('.')[0]
    text_file = open('../1-Vulnerability_Detection/code_static_analysis/'+file, "r", encoding='utf-8')
    data = text_file.read()
    name = guess.language_name(data)
        
    if(name=="C"):
        shutil.copy('../1-Vulnerability_Detection/code_static_analysis/'+file, 'to_check/'+name_to_save+".c")

In [14]:
repositories_final

,name
0,google/brotli
1,gentilkiwi/mimikatz
2,octalmage/robotjs
3,openresty/openresty
4,jedisct1/libsodium
...,...
128,notepad-plus-plus/notepad-plus-plus
129,apple/swift
130,pytorch/pytorch
131,onnx/onnx


In [27]:
with_clone = dict()
idx = 0
i = 133

name_repo = repositories_final['name'][i]
print(name_repo)
proc = subprocess.run(["curl", "https://api.github.com/repos/"+name_repo+"/tags"], capture_output=True)
hash_tags = []
json_file = json.loads(proc.stdout.decode('utf-8'))

for entry in json_file:
    hash_tags.append(entry['commit']['sha']) 
folder = name_repo.split('/')[-1]
repo = git.Repo.clone_from('https://github.com/'+name_repo+'.git', folder, no_checkout=True)   

for release in hash_tags:
    repo.git.checkout(release)

    print("Execute nicad for each file for release "+release)
    
    p = Popen(['C:/cygwin64/bin/bash.exe', '-c', '. /etc/profile; '+
   'nicad6cross functions c C:/Users/Grazia/Desktop/Tesi/StackOverflow_Vulnerabilities/2-Vulnerabilty_Propagation/to_check '+ 
               'C:/Users/Grazia/Desktop/Tesi/StackOverflow_Vulnerabilities/2-Vulnerabilty_Propagation/'+folder], 
      stdout=PIPE, stderr=STDOUT)
    p.communicate()[0]

    if os.path.exists('to_check_functions-blind-crossclones/to_check_functions-blind-crossclones-0.30.xml'):
        tree = ET.parse('to_check_functions-blind-crossclones/to_check_functions-blind-crossclones-0.30.xml')
        root = tree.getroot()
        lista = []
        for clones in root.findall('clone'):
            for source in clones:
                lista.append(source.attrib['file'])
        
        if len(root.findall('clone'))>0:
            with_clone[idx] = {'repo': folder, 'files': set(lista), 'release': release}
            idx = idx+1

        shutil.rmtree('to_check_functions-blind-crossclones')
        all_files = os.listdir()

        for item in all_files:
            if item.endswith(".log"):
                os.remove(item)

    all_files = os.listdir()
    for item in all_files:
        if item.startswith(folder) and item.endswith('.xml'):
            os.remove(item)
os.system('rmdir /S /Q "{}"'.format(folder))
print("Done for "+folder)

In [24]:
with_clone

{0: {'repo': 'fish-shell',
  'files': {'C:/Users/Grazia/Desktop/Tesi/StackOverflow_Vulnerabilities/2-Vulnerabilty_Propagation/fish-shell/event.c',
   'C:/Users/Grazia/Desktop/Tesi/StackOverflow_Vulnerabilities/2-Vulnerabilty_Propagation/fish-shell/history.c',
   'C:/Users/Grazia/Desktop/Tesi/StackOverflow_Vulnerabilities/2-Vulnerabilty_Propagation/fish-shell/input_common.c',
   'C:/Users/Grazia/Desktop/Tesi/StackOverflow_Vulnerabilities/2-Vulnerabilty_Propagation/fish-shell/kill.c',
   'C:/Users/Grazia/Desktop/Tesi/StackOverflow_Vulnerabilities/2-Vulnerabilty_Propagation/fish-shell/reader.c',
   'C:/Users/Grazia/Desktop/Tesi/StackOverflow_Vulnerabilities/2-Vulnerabilty_Propagation/fish-shell/sanity.c',
   'C:/Users/Grazia/Desktop/Tesi/StackOverflow_Vulnerabilities/2-Vulnerabilty_Propagation/fish-shell/wutil.c',
   'C:/Users/Grazia/Desktop/Tesi/StackOverflow_Vulnerabilities/2-Vulnerabilty_Propagation/fish-shell/xdgmime.c',
   'C:/Users/Grazia/Desktop/Tesi/StackOverflow_Vulnerabilities/2

In [25]:
name = repositories_final['name'][i].replace('/','_')
name

'fish-shell_fish-shell'

In [26]:
if with_clone:
    with open('dicts_c/'+name+".pkl", 'wb') as f:
        pickle.dump(with_clone, f)